In [13]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time
from datetime import datetime

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

import multiprocessing
cpu_cores = multiprocessing.cpu_count()

In [3]:
uber = pd.read_csv('/Users/austinwhaley/Desktop/DSI-SF-4-austinmwhaley/other_datasets/uber_data2.csv').drop('Unnamed: 0', 1)
uber.head()

,Year,Month,Day,Hour,1,2,3,4,5,6,...,264,265,Dew PointF,Gust SpeedMPH,Humidity,Sea Level PressureIn,TemperatureF,VisibilityMPH,Wind SpeedMPH,WindDirDegrees
0,2015,2,1,0,0.0,0.0,4.0,39.0,0.0,0.0,...,0.0,0.0,6.1,0.0,55.0,30.19,19.9,10.0,0.0,0
1,2015,2,1,1,0.0,0.0,0.0,58.0,0.0,0.0,...,0.0,0.0,6.1,0.0,53.0,30.19,21.0,10.0,3.5,250
2,2015,2,1,2,0.0,0.0,0.0,30.0,0.0,0.0,...,0.0,0.0,6.1,0.0,51.0,30.19,21.9,10.0,3.5,0
3,2015,2,1,3,0.0,0.0,0.0,23.0,0.0,0.0,...,0.0,1.0,7.0,0.0,50.0,30.19,23.0,10.0,3.5,0
4,2015,2,1,4,0.0,0.0,0.0,11.0,0.0,0.0,...,0.0,0.0,8.1,0.0,51.0,30.19,24.1,10.0,0.0,0


In [5]:
from sklearn.preprocessing import MinMaxScaler

In [6]:
#Scale data between 0,1 for LSTM/GRU
MMS = MinMaxScaler()
uber = pd.DataFrame(MMS.fit_transform(uber), columns=uber.columns)

#  Remember to shift the data so we are predicting one time step future
X = uber.iloc[:-1,:]
y = uber.iloc[1:,4:269] #265 locationIDs
#y = uber.ix[:, 4:] #Non_Weather Set

# Train-Test-Split-80/20
X_train, X_test = X[:int(round(len(X)*0.8, 0))], X[int(round(len(X)*0.8, 0)):]
y_train, y_test = y[:int(round(len(y)*0.8, 0))], y[int(round(len(y)*0.8, 0)):]

#Check data validity
print len(X_train) + len(X_test), len(y_train) + len(y_test)

3598 3598


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [30]:
start_time = time.time()
print 'Executing ||', str(datetime.now()), '\n'

LR = LinearRegression(n_jobs=cpu_cores-1).fit(X_train, y_train)
#train_scores = cross_val_score(LR, X_train, y_train, cv=10, verbose=0, n_jobs=cpu_cores - 1)
#print 'train_scores_mean =', np.mean(train_scores)

print 'train_score =', LR.score(X_train, y_train)
print 'test_score =', LR.score(X_test, y_test)
print 'Runtime =', round((time.time() - start_time), 2), 'seconds'

Executing || 2017-01-21 12:05:06.954230 

train_score = 0.548645594969
test_score = 0.366275832198
Runtime = 0.27 seconds


In [28]:
print LR.coef_

[[  2.89970568e+10  -7.19838859e-05   8.04416409e-04 ...,   2.90809835e-03
    2.56289057e-03   3.87812591e-03]
 [ -1.06608336e+09  -4.94271239e-03  -1.36303466e-03 ...,  -2.94476274e-03
   -3.03418989e-03  -1.54493951e-03]
 [ -3.54276825e+11   1.00223273e-02  -6.10388904e-03 ...,  -4.13720725e-03
    3.54085871e-02  -1.46556991e-03]
 ..., 
 [ -4.66663303e+11   3.08724959e-02   1.85741305e-03 ...,  -1.98210409e-03
    1.47765226e-02   8.05639001e-03]
 [  8.73668321e+10  -9.80553219e-03  -7.17520832e-04 ...,  -1.65586981e-03
    6.45555473e-04  -3.20985238e-03]
 [ -9.17160833e+11   8.56977453e-03   8.96225589e-03 ...,  -2.09958786e-02
   -2.28010459e-02   1.14500883e-02]]
